<a href="https://colab.research.google.com/github/primojr/estudos_python/blob/master/py/rede_neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Objetivo:

## Treinar o modelo usando `MLPClassifier` e comprar o resultado do `randon_forest`


## 1.Importar bibliotecas 

In [1]:
from sklearn import datasets 
from sklearn.neural_network import  MLPClassifier 

# # Bibliotecas Auxiliares
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(13)

df = pd.read_csv("dfFAmostra.csv", delimiter=";")

# # TensorFlow e tf.keras
# import tensorflow as tf
# from tensorflow import keras


# print(tf.__version__)

In [ ]:
df.head(2)

,id,system,sexo_f,class_cep,t.idade_ate30,t.br,t.inpp,t.offus,t.b2w,t.lasa,t.ampl_rec,t.ampl_app,t.frente_offus_inapp,t.band_outras,classe1
0,e281e652-c369-4a8b-9a36-6e085cfcfca5,android,1,B.Neutro,0,0,0,0,0,1,-5.225747,-1.120118,0,0,SemCartaoApp
1,f987742c-4e4f-418b-a95d-2fedba305f4f,android,0,B.Neutro,0,0,0,1,0,0,-6.654153,2.538974,0,0,ComCartaoApp


# 2. Ajustes na base

In [2]:
# Sys
df['system'] = df['system'].replace('android','1')
df['system'] = df['system'].replace('ios','0')
df['system'] = df['system'].astype(float)

# # Classe cep
df['class_cep'].value_counts()
df['class_cep'] = df['class_cep'].replace('A.Ruim','0')
df['class_cep'] = df['class_cep'].replace('B.Neutro','1')
df['class_cep'] = df['class_cep'].replace('C.Medio','2')
df['class_cep'] = df['class_cep'].replace('D.Bom','3')
df['class_cep'] = df['class_cep'].replace('E.MuitoBom','4')
df['class_cep'] = df['class_cep'].astype(float)

# Remover na()
df = df.dropna()
df = df.reset_index()

#I Tipos dos dados
df.dtypes

index                     int64
id                       object
system                  float64
sexo_f                    int64
class_cep               float64
t.idade_ate30             int64
t.br                      int64
t.inpp                    int64
t.offus                   int64
t.b2w                     int64
t.lasa                    int64
t.ampl_rec              float64
t.ampl_app              float64
t.frente_offus_inapp      int64
t.band_outras             int64
classe1                  object
dtype: object

# 3. Separar amostra e treino

In [3]:
#df = df.loc[:,df.columns[1:]]
X = df.loc[:, df.columns[2:14]]
y = df.loc[:,['classe1']]

n_aleatorio = np.arange(df.shape[0])
np.random.shuffle(n_aleatorio)

X_train = X.loc[n_aleatorio[:3000],:]
X_test  = X.loc[n_aleatorio[3000:],:]

y_train = y.loc[n_aleatorio[:3000],:]
y_test  = y.loc[n_aleatorio[3000:],:]


# 4. Treino do modelo

- **Paramentros:**
  - `hidden_layer_sizes` = 
  - `activation` = Função de ativação
  - `max_inter` = Maximo de interação
  - `learning_rate_init` = 



In [32]:
from pandas.core.common import random_state
clf = MLPClassifier(solver = 'adam',
                    hidden_layer_sizes=(2,) ,
                    learning_rate_init=0.00001, 
                    activation='logistic',
                    max_iter = 1500,
                    random_state = 1
                    )

clf.fit(X_train, y_train)

y_pred = clf.predict_proba(X_test)
y_pred

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[0.68800329, 0.31199671],
       [0.65993623, 0.34006377],
       [0.6498373 , 0.3501627 ],
       ...,
       [0.68566716, 0.31433284],
       [0.61285889, 0.38714111],
       [0.63827282, 0.36172718]])

# 5. Validar ajuste

In [33]:
# Categorizar a Classe de previsão
y_linha = pd.DataFrame(y_pred)

y_linha.loc[y_linha[1] >= .5, 'Class'] = 'ComCartaoApp'
y_linha.loc[y_linha[1] <  .5, 'Class'] = 'SemCartaoApp' 
  
# y_linha['Class'].value_counts()
#print(type(y_pred), type(y_test))

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_linha['Class'])

0.5489078822412156